In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import keras.layers as L
from keras import optimizers, losses, metrics, Model
from keras.callbacks import EarlyStopping
from keras.models import Sequential, load_model
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import tensorflow_model_optimization as tfmot
import tensorflow_addons as tfa

C:\Users\Umer\AppData\Roaming\Python\Python39\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\Umer\AppData\Roaming\Python\Python39\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an i

In [2]:
inputPath = r"C:\Rotated Equal Data"
inputPath = r"D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\Equal Data"

commands = np.array(tf.io.gfile.listdir(str(inputPath)))
print(commands)
N_CLASSES = len(commands)
print(N_CLASSES)

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

filesArray = []
for path, subdirs, files in os.walk(inputPath):
    for name in files:
        filesArray.append(os.path.join(path + "\\", name))
print(len(filesArray))



#for i in filesArray:
    #print(i)
    
max_length = 1000
df = []
labels = []
for filename in filesArray:
    data = np.loadtxt(filename, dtype=np.float32)
    
    data = data/2 #Normalise

    if len(data) < max_length:
        # pad the time series with zeros to a length of 1000
        data = np.pad(data, [(0, max_length - len(data)), (0, 0)], mode='constant')
    elif len(data) > max_length:
        # truncate the time series to a length of 1000
        data = data[:max_length, :]
    a = filename.split("\\")[-2]
    if a == "True":
        label = [1,0]
    else:
        label = [0,1]
    labels.append(label)

    data = np.expand_dims(data, axis=-1)
    data = np.expand_dims(data, axis=-1)
    df.append(data)

df_array = np.array(df)
labels_array = np.array(labels)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)

print(df_array.shape)
print(labels_array.shape)

['False' 'True']
2
1810
(1810, 1000, 1, 1)
(1810, 2)


In [3]:
df_array[1].shape

(1000, 1, 1)

In [5]:
import datetime

# Define the log directory for TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

def lr_schedule(epoch, lr):
    if epoch < 5:
        return lr
    else:
        print(0.001 * tf.math.exp(-0.02*((epoch-5))))
        return 0.001 * tf.math.exp(-0.02*((epoch-5)))

# Define the TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

    
total_epochs = 0

model = Sequential()

model.add(L.Input(shape=(1000,1, 1)))

# Block 1
model.add(L.Conv2D(8, (100, 1), activation='relu'))
model.add(L.AveragePooling2D((2, 1), strides=(2, 1)))

# Block 2
model.add(L.Conv2D(16, (15, 1), activation='relu'))
model.add(L.AveragePooling2D((2, 1)))

# Block 3
model.add(L.Conv2D(16, (5, 1), activation='relu'))
model.add(L.AveragePooling2D((2, 1)))

# Block 4
model.add(L.Conv2D(16, (3, 1), activation='relu'))
model.add(L.AveragePooling2D((2, 1)))

# Fully connected L
model.add(L.Flatten())
model.add(L.Dense(32, activation='relu'))
model.add(L.Dense(2, activation='sigmoid'))

model.compile(optimizer=tf.optimizers.SGD(learning_rate = 0.001, momentum=0.9),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])

# model.build((235, 4001, 1))
print(model.summary())

new_epochs = 4
model.fit(df_array, labels_array, shuffle=1,  initial_epoch=total_epochs, epochs=total_epochs+new_epochs, validation_split=0.05, batch_size = 2, callbacks=[tensorboard_callback,lr_scheduler])
total_epochs += new_epochs

model.compile(optimizer=tf.optimizers.Adam(learning_rate = 0.001),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])

new_epochs = 10
model.fit(df_array, labels_array, shuffle=1,  initial_epoch=total_epochs, epochs=total_epochs+new_epochs, validation_split=0.05, batch_size = 32, callbacks=[tensorboard_callback,lr_scheduler])
total_epochs += new_epochs

model.evaluate(df_array, labels_array, batch_size=1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 901, 1, 8)         808       
                                                                 
 average_pooling2d_4 (Averag  (None, 450, 1, 8)        0         
 ePooling2D)                                                     
                                                                 
 conv2d_5 (Conv2D)           (None, 436, 1, 16)        1936      
                                                                 
 average_pooling2d_5 (Averag  (None, 218, 1, 16)       0         
 ePooling2D)                                                     
                                                                 
 conv2d_6 (Conv2D)           (None, 214, 1, 16)        1296      
                                                                 
 average_pooling2d_6 (Averag  (None, 107, 1, 16)      

[0.001769814407452941, 0.9993928074836731]

In [11]:
model.evaluate(df_array, labels_array, batch_size=1)

1810/1810 [==============================] - 2s 864us/step - loss: 0.0035 - categorical_accuracy: 0.9983


[0.003537428565323353, 0.9983425140380859]

In [6]:
tf.keras.models.save_model(model, 'model.h5')

In [ ]:
# Load your Keras model
model = tf.keras.models.load_model('model.h5')

# Define the pruning parameters
pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(
    initial_sparsity=0.1,
    final_sparsity=0.99,
    begin_step=0,
    end_step=19
)

# Define the pruning callback
pruning_callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir='/tmp')
]

# Create a pruned model
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, pruning_schedule=pruning_schedule)

# Train the pruned model
pruned_model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.000001),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])
pruned_model.fit(df_array, labels_array, shuffle=1, epochs=20, validation_split=0.25, callbacks=pruning_callbacks)

# Remove the pruning wrappers from the model
pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Save the pruned model
pruned_model.save('pruned_model.h5')

In [14]:
# Load your Keras model
pruned_model = tf.keras.models.load_model('pruned_model.h5')
pruned_model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.0001),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])
pruned_model.evaluate(df_array, labels_array, batch_size=1)

1810/1810 [==============================] - 1s 684us/step - loss: 0.0032 - categorical_accuracy: 0.9983


[0.0031585711985826492, 0.9983425140380859]

In [98]:
######### NON QUANTIZED ############
pruned_model = tf.keras.models.load_model("pruned_model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
######### QUANTIZED ############
pruned_model = tf.keras.models.load_model("pruned_model.h5")

def representative_data_gen():
    for input_value in df_array:
        input_value = np.expand_dims(input_value, axis=0)
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT, tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.float16]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

input_min = -1.0
input_max = 1.0
converter.default_ranges_stats = (input_min, input_max)

tflite_model = converter.convert()

with open('model_quantized.tflite', 'wb') as f:
    f.write(tflite_model)


In [3]:
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

In [4]:
# Get the input tensor details
input_details = interpreter.get_input_details()
input_tensor_index = input_details[0]['index']
output_details = interpreter.get_output_details()


# Get the scale and zero-point values from the input tensor
input_tensor_scale = input_details[0]['quantization_parameters']['scales']
input_tensor_zero_point = input_details[0]['quantization_parameters']['zero_points']

output_tensor_scale = output_details[0]['quantization'][0]
output_tensor_zero_point = output_details[0]['quantization'][1]

print("Input tensor scale:", input_tensor_scale)
print("Input tensor zero-point:", input_tensor_zero_point)

print("Output tensor scale:", output_tensor_scale)
print("Output tensor zero-point:", output_tensor_zero_point)

Input tensor scale: []
Input tensor zero-point: []
Output tensor scale: 0.0
Output tensor zero-point: 0


In [ ]:
################## For Quantized only ###################
for j in range(len(df_array)):
    for i in range(len(df_array[0])):
        df_array[j][i][0] = int(df_array[j][i][0] / input_tensor_scale) + input_tensor_zero_point

In [18]:
i = 21

for i in range(1000):
    df_array[0][i][0][0] = 0.5

interpreter.set_tensor(input_details[0]['index'], np.expand_dims(df_array[0], axis=0))
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
print(output)

[[0.021377  0.9999674]]


In [10]:
correct = 0
total = 0
from scipy.stats import pearsonr


for i in range(len(df_array)):
    interpreter.set_tensor(input_details[0]['index'], df_array[i:i+1])
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    #origional = model.predict(df_array[i:i+1], verbose=0)
        
    print(f'Label: {labels_array[i]}  tflite model prediction: {output}')


Label: [0 1]  tflite model prediction: [[1.6389e-13 1.0000e+00]]
Label: [0 1]  tflite model prediction: [[2.6082763e-17 1.0000000e+00]]
Label: [1 0]  tflite model prediction: [[1.         0.34204093]]
Label: [1 0]  tflite model prediction: [[0.9991882  0.68220365]]
Label: [0 1]  tflite model prediction: [[0. 1.]]
Label: [0 1]  tflite model prediction: [[9.039549e-10 1.000000e+00]]
Label: [1 0]  tflite model prediction: [[1.         0.10047024]]
Label: [1 0]  tflite model prediction: [[1.         0.00131643]]
Label: [1 0]  tflite model prediction: [[1.0000000e+00 2.1977555e-09]]
Label: [0 1]  tflite model prediction: [[3.1639827e-17 1.0000000e+00]]
Label: [1 0]  tflite model prediction: [[0.999998   0.05570659]]
Label: [1 0]  tflite model prediction: [[9.879068e-01 4.923059e-06]]
Label: [0 1]  tflite model prediction: [[2.6082763e-17 1.0000000e+00]]
Label: [0 1]  tflite model prediction: [[1.6972016e-14 1.0000000e+00]]
Label: [1 0]  tflite model prediction: [[0.9914367  0.00120506]]
Lab